In [1]:
import torch
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score, recall_score, f1_score
from NN_Models import *
import json
import numpy as np


def print_scores(y_test, y_pred):
    print('='*25 + 'Evaluation results' + '='*25)
    print('The accuracy score of prediction is : {}'.format(accuracy_score(y_test, y_pred)))
    print('The recall   score of prediction is : {}'.format(recall_score(y_test, y_pred, average='weighted')))
    print('The f1       score of prediction is : {}'.format(f1_score(y_test, y_pred, average='weighted')))


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X_train = torch.load('data/X_author_train.pt')
y_train = torch.load('data/y_train.pt')

# X_train = torch.load('data/X_author_all.pt')
# y_train = torch.load('data/y_all.pt')

X_test = torch.load('data/X_author_test.pt')
y_test = torch.load('data/y_test.pt')

In [3]:
X_train.shape

torch.Size([6268, 21146])

In [4]:
X_test.shape

torch.Size([3086, 21146])

In [5]:
X = torch.concat((X_train, X_test), axis=0)
y = torch.concat((y_train, y_test), axis=0)

In [6]:
# from itertools import combinations

# edges = []

# for au in tqdm(range(y.shape[1])):
#     col = []
#     for i in range(y.shape[0]):
#         if y[i, au] == 1:
#             col.append(i)
    
#     for i, j in combinations(col, 2):
#         if [i, j] not in edges:
#             edges.append([i, j])
            
# edges = torch.tensor(np.array(edges).T)
# print(edges.shape)

In [7]:
# torch.save(edges, 'data/edges.pt')

edges = torch.load('data/edges.pt')
edges

tensor([[  28,   28,   28,  ..., 9260, 9260, 9295],
        [  58,   79,  158,  ..., 9295, 9352, 9352]])

In [8]:
# torch.save(edges, 'data/edges.pt')

In [9]:
# import torch_geometric
# from torch_geometric.utils import to_networkx

# dataset = torch_geometric.datasets.KarateClub()
# data = dataset.data

In [10]:
mask = []
for i in range(X.shape[0]):
    if i >= 6268:
        mask.append(False)
    else:
        mask.append(True)
mask = torch.tensor(mask)

In [11]:
mask

tensor([ True,  True,  True,  ..., False, False, False])

In [12]:
model = GraphNeuralNetwork()
model.fit(X, edges, y, mask)

In [13]:
model.train(epochs=10)

100%|██████████| 10/10 [03:54<00:00, 23.43s/it]


In [25]:
# logits = model.model(X_test, edges)
# edges.shape

logits = model.model(X, edges)[0][~mask]

In [15]:
logits.shape

torch.Size([3086, 100])

In [16]:
def get_author(matrix, threshold):
    result = []
    for i in tqdm(range(matrix.shape[0])):
        au = ""
        for j in range(100):
            if matrix[i, j] >= threshold:
                au += str(j) + " "
        if au:
            result.append(au[:-1])
        else:
            result.append("-1")
    return result

In [17]:
test_list = get_author(y_test, threshold=1)

100%|██████████| 3086/3086 [00:02<00:00, 1109.74it/s]


In [18]:
threshold = 7
pred_list = get_author(logits, threshold=threshold)

100%|██████████| 3086/3086 [00:03<00:00, 870.27it/s]


In [19]:
print_scores(test_list, pred_list)

=========================Evaluation results=========================
The accuracy score of prediction is : 0.1966947504860661
The recall   score of prediction is : 0.1966947504860661
The f1       score of prediction is : 0.06465947118605044


In [94]:
n = 0

print(train_list[n])
print(pred_list[n])

36 42
36 42


In [97]:
logits[0][0][42]

tensor(7.2446, grad_fn=<SelectBackward0>)

tensor(8.9808, grad_fn=<SelectBackward0>) --- 83
tensor(6.5715, grad_fn=<SelectBackward0>) --- 99
